In [1]:
import os
os.chdir('../')

### from "sample noise data pair by Euler and save pt.ipynb"

In [2]:
import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_NFE=120_N=256.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64])


In [12]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False

###############################################################################
# 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
###############################################################################
sys.argv = [
    "main.py",
    "--config", "imagenet64.yml",  # 사용하려는 config
    "--sample",
    "--sample_type", "rbfsolverglq10sepbeta",
    "--dpm_solver_type", "data_prediction",
    "--dpm_solver_order", "3",
    "--timesteps", "10",
    "--skip_type", "logSNR",
    "--log_scale_min", "-6.0",
    "--log_scale_max", "6.0",
    "--scale_dir", "/data/data/rbfsolverglq10sepbeta",
    "--ni"
]

###############################################################################
# 2) 인자/설정 로드
###############################################################################
args, config = parse_args_and_config()

###############################################################################
# 3) Diffusion 객체 생성 -> 모델 로딩
###############################################################################
diffusion = Diffusion(args, config, rank=0)
diffusion.prepare_model()
diffusion.model.eval()

###############################################################################
# 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
###############################################################################
device = diffusion.device


INFO - main.py - 2025-03-28 16:08:11,317 - Using device: cuda
INFO - main.py - 2025-03-28 16:08:11,317 - Using device: cuda
INFO - main.py - 2025-03-28 16:08:11,317 - Using device: cuda
INFO - main.py - 2025-03-28 16:08:11,317 - Using device: cuda


[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


In [13]:
!mkdir /data/data/rbfsolverglq10sepbeta

mkdir: cannot create directory ‘/data/data/rbfsolverglq10sepbeta’: File exists


In [14]:
n = 32
i = 0
while True:    
    start = i*n
    end = (i+1)*n
    if end > len(noise):
        break
    noise_batch = noise[start:end].to(device)
    target_batch = target[start:end].to(device)
    (pred, optimal_gammas), _ = diffusion.sample_image(noise_batch, diffusion.model, target=target_batch, exp_num=i)
    loss = F.mse_loss(pred, target_batch)
    print('loss :', loss)
    print(optimal_gammas)
    i += 1
    break


  0%|          | 0/10 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [03:41<00:24, 24.66s/it]

/data/data/rbfsolverglq10sepbeta/NFE=10,p=3,exp_num=0.npy  saved!
loss : tensor(0.0070, device='cuda:0')
tensor([[[ 1.3555,  0.0000,  0.0000,  0.0000],
         [ 0.1974, -1.0714,  0.0000,  0.0000],
         [ 2.4067,  2.1275,  2.3255,  0.0000],
         [ 4.1459, -1.8957,  4.3916,  0.0000],
         [ 2.2812,  1.9737,  2.2069,  0.0000],
         [ 3.8478, -2.2037,  4.1129,  0.0000],
         [ 0.8271, -4.2556,  0.7200,  0.0000],
         [ 0.5241, -0.0388,  0.4380,  0.0000],
         [ 5.2355,  5.4975,  0.0000,  0.0000],
         [ 3.0499,  0.0000,  0.0000,  0.0000]],

        [[ 5.4576,  4.5754,  0.0000,  0.0000],
         [ 1.9296,  3.1685,  1.4953,  0.0000],
         [ 0.8015,  3.8585,  0.8773, -1.4275],
         [ 1.3671,  3.8271,  0.8831, -0.9291],
         [ 0.5469,  1.7865,  0.8528, -4.5743],
         [ 1.1488, -2.4216,  0.8789, -3.0166],
         [ 0.9408, -3.0620,  0.6616, -4.1923],
         [ 1.1306, -2.7697,  1.1010, -5.2607],
         [ 5.9400, -3.9184,  5.3974,  0.0000],


### Batch

In [2]:
import sys
import torch
import numpy as np
from main import parse_args_and_config, Diffusion
from tqdm import tqdm

import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_NFE=120_N=256.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

model = None
for NFE in tqdm([5, 10, 15, 25]):
    sys.argv = [
        "main.py",
        "--config", "imagenet64.yml",  # 사용하려는 config
        "--sample",
        "--sample_type", "rbfsolverglq10sepbeta",
        "--dpm_solver_type", "data_prediction",
        "--dpm_solver_order", "3",
        "--timesteps", str(NFE),
        "--skip_type", "logSNR",
        "--log_scale_min", "-6.0",
        "--log_scale_max", "6.0",
        "--log_scale_num", "100",
        "--scale_dir", "/data/data/rbfsolverglq10sepbeta",
        "--exp_num", "0",
        "--verbose", "critical",
        "--ni",
    ]

    args, config = parse_args_and_config()
    diffusion = Diffusion(args, config, rank=0)

    if model is None:
        diffusion.prepare_model()
        diffusion.model.eval()
        model = diffusion.model
    device = diffusion.device

    n = 16
    noise_batch = noise[:n].to(device)
    target_batch = target[:n].to(device)
    (pred, _), _ = diffusion.sample_image(noise_batch, model, target=target_batch)
    print(F.mse_loss(pred, target_batch))


torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64])


  0%|          | 0/4 [00:00<?, ?it/s]

[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


 25%|██▌       | 1/4 [00:49<02:28, 49.67s/it]

/data/data/rbfsolverglq10sepbeta/NFE=5,p=3,exp_num=0.npy  saved!
tensor(0.0296, device='cuda:0')


 25%|██▌       | 1/4 [01:33<04:41, 93.81s/it]


_LinAlgError: torch.linalg.solve: The solver failed because the input matrix is singular.

In [9]:
print('done')

done
